In [ ]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
import plotly.io as pio
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

In [ ]:
from electropy.charge import Charge
from electropy.volume import *

layout = go.Layout(
    width=700,
    height=700,
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        ticks='',
        showticklabels=False
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        ticks='',
        showticklabels=False
    )
)

charge_val = 1e-7
position = 5
volume_dim = 10
z_plane = 0
field_scaling = 20
field_threshold = 15

h = 1

x, y = np.meshgrid(
    np.arange(-volume_dim, volume_dim + h, h),
    np.arange(-volume_dim, volume_dim + h, h)
)

time = np.linspace(0,101,101)

ut = np.empty([x.shape[0],y.shape[1],time.size])
vt = np.empty([x.shape[0],y.shape[1],time.size])

for dt in range(time.size):
    xpos = position-1
    ypos = -9+time[dt]*2*9/time.size

    charges = [Charge([ypos, xpos, 0], -charge_val/4)]

    charges.append(Charge([0, -position, 0], -charge_val))
    charges.append(Charge([0, +position, 0], +charge_val))

    # These two are "switched", not sure why numpy needs this. Needs clarification.
    u = np.squeeze(field(charges, z_range=[z_plane, z_plane], h = h, type="potential", component='y'))
    v = np.squeeze(field(charges, z_range=[z_plane, z_plane], h = h, type="potential", component='x'))

    # Scaling and thresholding the field so that the arrows fit nicely in the plot
    u=u/field_scaling
    v=v/field_scaling
    threshold = field_threshold

    nan_mask = np.isnan(u) + np.isnan(v)
    v[nan_mask] = 0
    u[nan_mask] = 0

    mask = (u**2+v**2)>threshold

    u[mask] = 0
    v[mask] = 0
    
    fig = ff.create_quiver(x, y, u, v,
                       scale=.25,
                       arrow_scale=0.3,
                       name='Electric field',
                       line=dict(width=1))

    # Create negative charge
    negative_charge = go.Scatter(x=[xpos], y=[ypos],
                        mode='markers',
                        marker=dict(size=6, color ='blue'),
                        name='Negative charge')

    # Add points to figure
    fig.add_trace(negative_charge)

    # Create negative charge
    negative_charge = go.Scatter(x=[-5], y=[0],
                        mode='markers',
                        marker=dict(size=24, color ='blue'),
                        name='Negative charge')

    # Add points to figure
    fig.add_trace(negative_charge)

    # Create negative
    positive_charge = go.Scatter(x=[5], y=[0],
                        mode='markers',
                        marker=dict(size=24, color ='red'),
                        name='Positive charge')

    # Add points to figure
    fig.add_trace(positive_charge)

    fig['layout'] = layout
    if dt<10:
        fig_name = 'fig_00'+str(dt)+'.png'
    elif dt<100:
        fig_name = 'fig_0'+str(dt)+'.png' 
    pio.write_image(fig, fig_name, format='png', width=700, height=700, scale=2)

In [ ]:
!tar cvfz zipname.tar.gz fig_*.png

# After this step, download the tarfile and run the two commands below on your local
#convert -delay 3 -layers Optimize fig_*.png -loop 0 field.gif
